## Non-Negative Matrix Factorization (NMF)

Utilizemos el diccionario creado y el corpus de prueba para ver cómo implementar un modelo, en este caso **Non-Negative matrix factorization**. Vamos a utlizar tanto la implementación en **Gensim** (que es más fácil para obtener información de tipo semántica) como la de **Scikit-learn** que nos permite a acceder a más información numérica.

Comenzamos cargando el diccionario y el corpus previamente construidos:

In [39]:
# -*- coding: utf8 -*-
from gensim.corpora import Dictionary
import cPickle as pk

# Cargado del diccionario construido
dictionary = Dictionary.load('tutorial.dict')

# Cargado del corpus como bag or words
corpus = pk.load(file('Tutorial_corpus.pk','r'))

Comenzaremos utlizando la implementación en **Scikit-learn**, para ello lo primero que tenemos que hacer es transformar el corpus como una matriz *sparse* que **Scikit-learn** entiende. **Gensim** nos permite hacer esto fácilmente:

In [40]:
from gensim.matutils import corpus2csc

# Corpus para sklearn
corpus2sklearn = corpus2csc(corpus).T

La transposición (*.T*) nos permite que la matriz quede armada de forma tal que cada documento sea un vector fila en el espacio de términos. Esto se ve inspeccionando las dimensiones de la matriz:

In [41]:
print('Dimensiones: {}'.format(corpus2sklearn.shape))

Dimensiones: (400, 18646)


Vemos que la matriz tiene 400 filas que es el número de documentos cargados y 18646 términos que los describen.

### NMF

**Non Negative Matrix Factorization** es un tipo de descomposición matricial que en la mayoría de los casos no puede hacer en forma exacta. La idea es poder descomponer una matriz con elemenos no negativos como un producto de otras dos matrices compuestas también por elementos no negativos: 

$$ A^{(m \times n)} = H^{(m \times k)} \cdot W^{(k \times n)} $$

donde $H$ y $W$ tienen todos sus elementos no negativos y $k$ es un parámetro que indica una dimensión latente y que debe ser elegido antes de realizar la descomposición.
**NMF** es particularmente útil para la descomposición de un corpus de texto en un cierto número de tópicos, especificados por el parámetro $k$. La no negatividad de todas las matrices involucradas en el cálculo permite que la interpretación sea directa: si $A$ es una matriz de documentos por términos, $H$ contiene a los documentos descritos en la base de tópicos y $W$ a los tópicos descritos en el espacio de términos.

Vamos a utilizar **Scikit-learn** para la aplicación de este modelo. 
El esquema de trabajo en **Scikit-learn** es generalmente el siguiente:
- Importar la clase del modelo que queremos utilizar.
- Creamos un objeto a partir de la clase con los parámetros que querramos.
- Llamamos al método *fit_transform* del objeto creado pasandole como argumentos los datos, en nuestro caso la matriz asociada al corpus.

Por lo tanto comenzamos importando la clase de NMF:

In [42]:
from sklearn.decomposition import NMF

Debido a que nuestro corpus de prueba fue preparado especialmente compuesto por 4 tópicos, veamos si **NMF** es capaz de realizar el etiquetado en forma correcta. Partimos entonces definiendo un objeto *nmf* con 4 tópicos a partir de la clase importada:

In [43]:
# Objeto nmf a partir de la clase NMF
nmf = NMF(n_components=4)

Transformamos la matriz del corpus a la nueva base:

In [44]:
corpus_transformed = nmf.fit_transform(corpus2sklearn)

Vemos que la dimensiones de la matriz corpus transformada es ahora el número de documentos por el número de las nuevas dimensiones:

In [46]:
print('Dimensiones: {}'.format(corpus_transformed.shape))

Dimensiones: (400, 4)


Las dimensiones de la matriz transformada nos indica que la matriz de corpus transformado es efectivamente la matriz $H$ de nuestro algoritmo. Por ejemplo si inspeccionamos el primer elemento:

In [47]:
print(corpus_transformed[0])

[0.32101068 0.00219835 0.         0.00639528]


Vemos que el primer documento esta descrito por pesos no negativos en el espacio de los 4 tópicos. Correctamente normalizado podemos entender este vector como una distribución de probabilidad en el espacio de términos. La normalización podemos llevarla a cabo mediante **Scikit-learn**:

In [48]:
from sklearn.preprocessing import Normalizer

# Creamos un normalizador con norma 1, 
# esto es la suma de las componentes de un vector da 1,
# no confundir con la norma euclídea.

norm = Normalizer('l1')
corpus_transformed = norm.fit_transform(corpus_transformed)

Volviendo a inspeccionar el primer elemento:

In [49]:
print(corpus_transformed[0])

[0.97392743 0.00666968 0.         0.01940289]


De esta manera vemos que el primer elemento está asociado al primer tópico hallado por el algoritmo.
En caso que querramos etiquetar cada documento con una única etiqueta basta hallar el elemento más grande de vector en el espacio de tópicos. Lo hacemos fácilmente con la función *argmax* de **Numpy**:

In [50]:
import numpy as np

labels_predicted = [np.argmax(d) for d in corpus_transformed]

print('Etiquetas: {}'.format(labels_predicted))

Etiquetas: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 0, 2, 2, 3, 2, 2, 3, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

###### Comparación de etiquetas

Del corpus de prueba conocemos el etiquetado esperado: son 400 documentos con 4 tópicos esperados, ordenados de 100 en 100. Creamos entonces una lista con las etiquetas verdaderas:

In [51]:
labels_true = [0] * 100 + [1] * 100 + [2] * 100 + [3] * 100
print(labels_true)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

Recordemos que en python el signo *+* utilizado con listas significa concatenación. Del mismo modo, multiplicar por 100 una lista implica en concatenar 100 veces es lista consigo misma.
Para ver la concordancia de las etiquetas predichas y las etiquetas esperadas vemos información mutua entre ellas, que vale 0 cuando las etiquetas están perfectamente descorrelacionadas, y 1 cuando la coincidencia es perfecta:

In [52]:
from sklearn.metrics import normalized_mutual_info_score

nmi = normalized_mutual_info_score(labels_predicted, labels_true)
print('Información mutua: {}'.format(nmi))

Información mutua: 0.898364619583


La información mutua es muy cercana a 1 por lo vemos que **NMF** detecta muy bien los tópicos presentes corpus.

##### Interpretación de tópicos

La información del significado de los tópicos quedó guardada en la matriz $W$ del algoritmo. Esta puede hallarse en el atributo *components_* del objeto *nmf*:

In [53]:
print('Dimensiones: {}'.format(nmf.components_.shape))

Dimensiones: (4, 18646)


Vemos que efectivamente *components_* es la matriz de los 4 tópicos descritas en el espacio de los términos originales. Las componentes más grandes de cada vector fila indicarán cuáles son los términos más relevantes a la hora de definir un tópico. Para ello debemos hacer dos cosas:

- ordenar los índices de las componentes de los vectores tópicos según el peso de cada componente.
- identificar qué término está asociado a cada índice

Vamos paso a paso trabajando por ahora con el primer tópico:

In [54]:
# Trabajo solo en el primer tópico
c = nmf.components_[0]

# El tamaño de c es el índice más alto (menos 1 en realidad ya que se cuenta desde 0)
m = len(c)
# Creo una lista que contengan todos los índices
l = range(m)
# Ordeno una lista que contenga los índices según el peso de cada componente ordenados de mayor a menor (reverse = True)
ordered_index_list = sorted(l, reverse = True, key = lambda x: c[x])

El argumento *key* es quizás el más difícil de entender pero el más útil: es el criterio con el cual ordena. Se lee como: *x* es un elemento de la lista que se quiere ordenar. Estamos ordenando índices con el criterio de que los ubique según el valor la componente evaluado en dicho índice (*c[x]*). El hecho que los ordene de mayor a menor según este criterio está espedificado en el argumento *reverse = True*.
Vemos cuáles son los primeros 10 índices:

In [55]:
print(ordered_index_list[:10])

[0, 445, 117, 342, 322, 474, 468, 411, 51, 367]


A qué términos están asociados estos índices lo vemos a traves del diccionario:

In [56]:
topic_terms = [dictionary[i] for i in ordered_index_list[:10]]

# Unimos la lista en un string
print(u', '.join(topic_terms))

aborto, vida, mujer, mujeres, ley, embarazo, debate, salud, derecho, persona


Iterando con el mismo procedimiento, obtengamos la descripción de todos los tópicos:

In [57]:
nt = 0 # Indice auxiliar
for c in nmf.components_:
    
    m = len(c)
    l = range(m)
    ordered_index_list = sorted(l, reverse = True, key = lambda x: c[x])
    topic_terms = [dictionary[i] for i in ordered_index_list[:10]]
    print(u'Tópico {}: '.format(nt) + u', '.join(topic_terms))

    nt += 1

Tópico 0: aborto, vida, mujer, mujeres, ley, embarazo, debate, salud, derecho, persona
Tópico 1: trump, presidente, estados, unidos, casa, blanca, acuerdo, donald, países, washington
Tópico 2: dólar, inflación, mercado, banco, cambio, central, suba, bcra, tasa, us
Tópico 3: boca, equipo, partido, copa, guillermo, river, libertadores, pavón, barros, schelotto


Que son efectivamente los tópicos que esperabamos.

In [ ]:
### Tfidf